In [38]:
from datasets import load_dataset
from src.utils import linearise_input
import csv
import re
from src.utils import linearise_input, convert_to_features, form_qa_input_output
import torch

In [3]:
dataset = load_dataset("james-burton/textual-explanations")

dataset = dataset.map(
        lambda x: linearise_input(x, 'ord_first', 100, True),
        ) 
train = dataset['train']

Using custom data configuration james-burton--textual-explanations-19ff8605823ae74a
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--textual-explanations-19ff8605823ae74a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 649.41it/s]
Loading cached processed dataset at /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--textual-explanations-19ff8605823ae74a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f5fc54edbbba4b55.arrow
Loading cached processed dataset at /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--textual-explanations-19ff8605823ae74a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-fc479ca87a0f997f.arrow
Loading cached processed dataset at /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--textual-explanations-19ff8605823ae74a

Lets do a classifcation problem, try to classifiy which features are mentioned given the questions and the data input

In [54]:
train['narration']

def find_mentioned_fts(data_row):
    matches = re.finditer(r"F\d+\b", data_row['narration'], re.MULTILINE)
    data_row['mentioned_fts'] = sorted(list(set([match.group() for match in matches])))
    options =  [f'F{i}' for i in range(1,51)]
    data_row['labels'] = [1 if option in data_row['mentioned_fts'] else 0 for option in options]
    return data_row

def convert_to_features_clf(batch, tokenizer, max_input_length=400, max_output_length=350):
    if type(batch['input'][0]) == list:
        input_encodings = [tokenizer(i, padding="max_length", truncation=True, max_length=max_input_length) for i in batch['input']]
        input_ids = [i['input_ids'] for i in input_encodings]
        attention_mask = [i['attention_mask'] for i in input_encodings]
    else:
        input_encodings = tokenizer(batch['input'], padding="max_length", truncation=True, max_length=max_input_length)
        input_ids = input_encodings['input_ids']
        attention_mask = input_encodings['attention_mask']
    encodings = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': batch['mentioned_fts_ohe'],
    }
    return encodings

In [55]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=50,problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

dataset = dataset.map(
        lambda x: linearise_input(x, 'ord_first', 50, True),
        ) 
dataset = dataset.map(
    lambda x: find_mentioned_fts(x),
)
dataset = dataset.map(
    lambda x: convert_to_features_clf(x, tokenizer, 512), 
    batched=True, load_from_cache_file=False
    )



loading configuration file config.json from cache at /home/james/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24":

In [57]:
training_args = TrainingArguments(
    output_dir="exps_mentioned",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/james/CodingProjects/Local_level_model_explanations/exps_mentioned is already a clone of https://huggingface.co/james-burton/exps_mentioned. Make sure you pull the latest changes with `repo.git_pull()`.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ft_num_to_name, classes_dict, mentioned_fts, unique_id, narration, narrative_questions, feature_nums, old2new_ft_nums, old2new_classes, narrative_id, sign, task_name, predicted_class, model_name, input, mentioned_fts_ohe, values. If ft_num_to_name, classes_dict, mentioned_fts, unique_id, narration, narrative_questions, feature_nums, old2new_ft_nums,

RuntimeError: result type Float can't be cast to the desired output type Long

In [44]:
dataset['train']['labels'][0]

AttributeError: 'list' object has no attribute 'shape'

In [8]:
from datasets import load_dataset
from src.utils import linearise_input
import re
from src.utils import linearise_input
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from transformers.trainer_callback import EarlyStoppingCallback
import yaml
import argparse
import wandb
import os
from tqdm import tqdm

def find_mentioned_fts(data_row):
    matches = re.finditer(r"F\d+\b", data_row['narration'], re.MULTILINE)
    data_row['mentioned_fts'] = sorted(list(set([match.group() for match in matches])))
    options =  [f'F{i}' for i in range(1,51)]
    data_row['labels'] = torch.tensor([1.0 if option in data_row['mentioned_fts'] else 0.0 for option in options])
    return data_row

def convert_to_features_clf(batch, tokenizer, max_input_length=400, max_output_length=350):
    if type(batch['input'][0]) == list:
        input_encodings = [tokenizer(i, padding="max_length", truncation=True, max_length=max_input_length) for i in batch['input']]
        input_ids = [i['input_ids'] for i in input_encodings]
        attention_mask = [i['attention_mask'] for i in input_encodings]
    else:
        input_encodings = tokenizer(batch['input'], padding="max_length", truncation=True, max_length=max_input_length)
        input_ids = input_encodings['input_ids']
        attention_mask = input_encodings['attention_mask']
    encodings = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': batch['labels'],
    }
    return encodings

    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

# parser = argparse.ArgumentParser()
# parser.add_argument('--config', type=str, default='test',
#                     help='Name of config from the the multi_config.yaml file')
config_type = 'test_2'#parser.parse_args().config

In [9]:
# import yaml file
with open('../configs/which_exps_default.yaml') as f:
    args = yaml.safe_load(f)

# Update default args with chosen config
if config_type != 'default':
    with open('../configs/which_exps_configs.yaml') as f:
        yaml_configs = yaml.safe_load_all(f)
        yaml_args = next(conf for conf in yaml_configs if conf['config'] == config_type)
    args.update(yaml_args)
    print(f'Updating with:\n{yaml_args}\n')
print(f'\n{args}\n')

model = AutoModelForSequenceClassification.from_pretrained(args['model_base'], num_labels=50,problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
dataset = load_dataset("james-burton/aug-text-exps")
dataset = dataset.map(
        lambda x: linearise_input(x, args['linearisation'], args['max_features']),
        ) 
dataset = dataset.map(
    lambda x: find_mentioned_fts(x),
)
dataset = dataset.map(
    lambda x: convert_to_features_clf(x, tokenizer, args['max_input_len']), 
    batched=True, load_from_cache_file=False
    )

# Fast dev run if want to run quickly and not save to wandb
if args['fast_dev_run']:
    args['num_epochs'] = 1
    args['tags'].append("fast-dev-run")
    dataset['train'] = dataset['train'].select(range(50))
    # dataset['test'] = dataset['test'].select(range(10))
    output_dir = os.path.join(args['output_root'], 'testing')
    print("\n######################    Running in fast dev mode    #######################\n")

# If not, initialize wandb
if not args['fast_dev_run']:
    wandb.init(
        project="Which exps mentioned classifier",
        tags=args['tags'],
        save_code=True, 
        config={'my_args/'+k: v for k, v in args.items()},
    )
    # wandb.log(args)
    os.environ['WANDB_LOG_MODEL'] = 'True'
    output_dir = os.path.join(args['output_root'], wandb.run.name)
    print(f'Results will be saved @: {output_dir}')



# Initialise training arguments and trainer
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=args['num_epochs'],
    per_device_train_batch_size=args['batch_size'],
    per_device_eval_batch_size=args['batch_size'],
    logging_steps=args['logging_steps'],
    do_train=args['do_train'],
    do_predict=args['do_predict'],
    resume_from_checkpoint=args['resume_from_checkpoint'],
    report_to="wandb" if not args['fast_dev_run'] else "none",
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=args['save_total_limit'],
    load_best_model_at_end=True,     
)
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    callbacks=[EarlyStoppingCallback(args['early_stopping_patience'])] if args['early_stopping_patience'] > 0 else [],
    compute_metrics=compute_metrics
)

loading configuration file ../models/exps_mentioned/splendid-jazz-1/checkpoint-4690/config.json
Model config BertConfig {
  "_name_or_path": "../models/exps_mentioned/splendid-jazz-1/checkpoint-4690",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "

Updating with:
{'config': 'test_2', 'fast_dev_run': True, 'model_base': '../models/exps_mentioned/splendid-jazz-1/checkpoint-4690'}


{'config': 'test_2', 'fast_dev_run': True, 'do_train': True, 'do_predict': True, 'tags': ['bert'], 'batch_size': 4, 'linearisation': 'ord_first', 'max_features': 50, 'model_base': '../models/exps_mentioned/splendid-jazz-1/checkpoint-4690', 'output_root': 'models/exps_mentioned/', 'max_input_len': 512, 'num_epochs': 10, 'early_stopping_patience': 3, 'logging_steps': 100, 'device': 'cuda', 'resume_from_checkpoint': False, 'predict_batch_size': 4, 'save_total_limit': 1, 'augmented_ds': False}



All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ../models/exps_mentioned/splendid-jazz-1/checkpoint-4690.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
loading configuration file config.json from cache at /home/james/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,



######################    Running in fast dev mode    #######################



In [27]:
preds, label_ids, metrics = trainer.predict(dataset['test'])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ft_num_to_name, input, feature_nums, old2new_ft_nums, values, mentioned_fts, model_name, task_name, narration, narrative_questions, predicted_class, sign, unique_id, old2new_classes, narrative_id, classes_dict. If ft_num_to_name, input, feature_nums, old2new_ft_nums, values, mentioned_fts, model_name, task_name, narration, narrative_questions, predicted_class, sign, unique_id, old2new_classes, narrative_id, classes_dict are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 47
  Batch size = 4
100%|██████████| 12/12 [00:00<00:00, 21.63it/s]


In [28]:
preds

array([[ 0.52723694,  0.4932382 ,  0.42247894, ..., -8.340782  ,
        -8.164411  , -7.5624967 ],
       [-0.37740391, -0.3190624 , -0.543151  , ..., -7.8313503 ,
        -7.282925  , -6.986225  ],
       [ 0.63291717,  0.8955558 ,  1.1990858 , ..., -7.574162  ,
        -6.988083  , -7.6330614 ],
       ...,
       [-0.2679088 ,  0.2675999 , -0.09550278, ..., -7.8705893 ,
        -7.1796474 , -7.143261  ],
       [ 1.2595031 ,  1.0931427 ,  0.87521195, ..., -7.7288585 ,
        -7.3062162 , -7.615298  ],
       [ 3.5122275 ,  3.4347904 ,  3.348819  , ..., -6.1694584 ,
        -6.167973  , -5.503597  ]], dtype=float32)